In [1]:
from matplotlib import pyplot as plt
import numpy as np
from time import time
from qiskit.quantum_info.operators import Operator
from qiskit.providers.aer.pulse_new.de.DE_Problems import BMDE_Problem
from qiskit.providers.aer.pulse_new.de.DE_Solvers import BMDE_Solver
from qiskit.providers.aer.pulse_new.de.DE_Options import DE_Options
from qiskit.providers.aer.pulse_new.models.signals import VectorSignal, Constant, Signal
from qiskit.providers.aer.pulse_new.models.operator_models import OperatorModel

X = Operator.from_label('X')
Y = Operator.from_label('Y')
Z = Operator.from_label('Z')

# 1. Basic BMDE Problem/Solver interface

A `BMDE_Problem` represents a differential equation of the form $\dot{y}(t) = G(t) y(t)$, where currently it is assumed that $G(t)$ is specified by an `OperatorModel`. It is the core type of BMDE specification from which others will be built. E.g. the Schrodinger equation for a state vector will be a subclass of `BMDE_Problem`, accepting a `HamiltonianModel` and converting it into the proper generator.

Construct an `OperatorModel` for the generator:

In [2]:
r = 0.2
w = 5.
signals = [Constant(1.), Signal(1., w)]
operators = [-1j * 2 * np.pi * w * Z/2, -1j * 2 * np.pi * r * X/2]

generator = OperatorModel(operators=operators, signals=signals)

Construct the problem with the generator, initial time, and initial state.

In [3]:
#y0 = np.sqrt(0.5)*np.array([1., 1.])
y0 = np.array([1., 0.])
#y0 = np.eye(2)

bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0)

Construct the solver with the problem and some options.

In [4]:
options = DE_Options(atol=1e-6, rtol=1e-6)
solver = BMDE_Solver(bmde_problem, options=options)

Integrate up to some time. Here the time is chosen to implement a $\pi$-pulse.

In [5]:
solver.integrate(1. / r)

Look at the state.

In [6]:
#y_orthog = np.sqrt(0.5)*np.array([1., -1.])
#y_orthog = np.array([0., 1.])

#np.abs( ( y_orthog *solver.y).sum())**2
solver.y

array([-1.92710815e-05-0.00499853j, -7.60519598e-09+0.99999394j])

# 2. Specifying solving frames

When solving a BMDE, it can be numerically advantageous to solve in a different frame. A frame to solve the system in can be specified when constructing a `BMDE_Problem` via the keyword argument `frame_operator`. The default value, `frame_operator='auto'`, lets the `BMDE_Problem` class automatically choose the frame to solve in. Currently, it will automatically choose the anti-Hermitian part of the drift. We can set different values for `frame_operator` and see how long it takes to solve the system.

Setting `frame_operator=None` forces the problem to be solved in the frame it is specified in. 

In [7]:
bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0, frame_operator=None)
options = DE_Options(atol=1e-8, rtol=1e-8)
solver = BMDE_Solver(bmde_problem, options=options)

start = time()
solver.integrate(1. / r)
print('Time taken: ' + str(time() - start))
print(solver.y)

Time taken: 0.4456298351287842
[-1.96262663e-05-0.00500051j -4.77089104e-10+0.9999871j ]


Next, solve the system in the automatically determined frame

In [8]:
bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0, frame_operator='auto')
options = DE_Options(atol=1e-8, rtol=1e-8)
solver = BMDE_Solver(bmde_problem, options=options)

start = time()
solver.integrate(1. / r)
print('Time taken: ' + str(time() - start))
print(solver.y)

Time taken: 0.2699251174926758
[-1.96352698e-05-0.00500054j  2.82321676e-10+0.99998751j]


Above we see a speed up of nearly a factor of $2$. The scale of the time differences largely depends on parameters of the system.

# 3. Solving with cutoff frequencies (Rotating wave approximation)

In addition to specifying frame, we may also specify a cutoff frequency. The standard cutoff frequency for this model, the rotating wave approximation, is `2 * w`.

In [9]:
bmde_problem = BMDE_Problem(generator=generator, t0=0., y0=y0, cutoff_freq=2*w)
options = DE_Options(atol=1e-8, rtol=1e-8)
solver = BMDE_Solver(bmde_problem, options=options)

start = time()
solver.integrate(1. / r)
print('Time taken: ' + str(time() - start))
print(solver.y)

KeyError: 'rhs'

Observe: solving with a cutoff frequency increases the speed of solving dramatically, albeit at the expense of a loss of accuracy. Again, the scale of the speed gains and accuracy loss will depend on the parameters of the model.

# 4. Specifying different numerical methods

The default method is `scipy`'s RK45